In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


# Cases, fine age split

In [2]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')
dfAge.tail(10)


,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive
804,2021-W43,80+,5419,91.0,"19,0",15711,263.0
805,2021-W44,0-2,3447,103.0,"6,0",6324,189.0
806,2021-W44,3-5,13730,279.0,"0,5",25698,522.0
807,2021-W44,6-11,23964,939.0,"2,2",88230,3459.0
808,2021-W44,12-15,17732,305.0,"1,5",48333,830.0
809,2021-W44,16-19,12651,217.0,"0,4",34664,595.0
810,2021-W44,20-39,13792,273.0,"5,2",204320,4046.0
811,2021-W44,40-64,17148,240.0,"5,6",324182,4538.0
812,2021-W44,65-79,9166,153.0,"10,5",82773,1382.0
813,2021-W44,80+,5919,105.0,"23,6",17283,306.0


In [3]:
curYears = dfAge.Uge.apply(lambda x: int(x[:4]))

# Remove everything before 2021
curdf = dfAge[curYears > 2020].copy()


weekNums = curdf.Uge.apply(lambda x: int(x[-2:]))


# Uge 36: 07-09-2021 (tirsdag) og 12-09-2021 (søndag)

weekOffset = weekNums - 36
# allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-07') + np.timedelta64(x*7,'D'))
allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-12') + np.timedelta64(x*7,'D'))

allDates
curdf['Dato'] = allDates

In [4]:


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Antal positive'].values
    toAdd = thisdf['Antal positive'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Nye smittetilfælde\n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_Cases')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Antal positive'].values
    # toAdd = thisdf['Antal positive'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt nye smittetilfælde')
ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_CasesWeekly')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Positive pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal positive'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt nye smittetilfælde per 100.000')
# ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_CasesWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testede

In [7]:
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
809,2021-W44,16-19,12651,217.0,"0,4",34664,595.0,2021-11-07
810,2021-W44,20-39,13792,273.0,"5,2",204320,4046.0,2021-11-07
811,2021-W44,40-64,17148,240.0,"5,6",324182,4538.0,2021-11-07
812,2021-W44,65-79,9166,153.0,"10,5",82773,1382.0,2021-11-07
813,2021-W44,80+,5919,105.0,"23,6",17283,306.0,2021-11-07


In [8]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal testede'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal testede'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal testede'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal testede'],label=curAge)
    toAdd = thisdf['Antal testede'].values
    toAdd = thisdf['Antal testede'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Testede \n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af alle testede')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_Testede')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
809,2021-W44,16-19,12651,217.0,"0,4",34664,595.0,2021-11-07
810,2021-W44,20-39,13792,273.0,"5,2",204320,4046.0,2021-11-07
811,2021-W44,40-64,17148,240.0,"5,6",324182,4538.0,2021-11-07
812,2021-W44,65-79,9166,153.0,"10,5",82773,1382.0,2021-11-07
813,2021-W44,80+,5919,105.0,"23,6",17283,306.0,2021-11-07


In [10]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Testede pr. 100.000 borgere'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Testede pr. 100.000 borgere'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Testede pr. 100.000 borgere'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal Testede'],label=curAge)
    toAdd = thisdf['Testede pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal Testede'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt testede per 100.000')
# ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax1.set_ylim(top = 25000)
# ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_TestedeWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
print(curdf.tail(9).sum())
display(curdf.tail(9))
print((6324+25698+ 88230)/831807)
print((189.0+522.0+ 3459.0)/15867.0)
# Testede pr. 100.000 borgere

Uge                               2021-W442021-W442021-W442021-W442021-W442021-W...
Aldersgruppe                                 0-23-56-1112-1516-1920-3940-6465-7980+
Testede pr. 100.000 borgere                                                  117549
Positive pr. 100.000 borgere                                                 2614.0
Nyindlagte pr. 100.000 borgere                        6,00,52,21,50,45,25,610,523,6
Antal testede                                                                831807
Antal positive                                                              15867.0
dtype: object


,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
805,2021-W44,0-2,3447,103.0,"6,0",6324,189.0,2021-11-07
806,2021-W44,3-5,13730,279.0,"0,5",25698,522.0,2021-11-07
807,2021-W44,6-11,23964,939.0,"2,2",88230,3459.0,2021-11-07
808,2021-W44,12-15,17732,305.0,"1,5",48333,830.0,2021-11-07
809,2021-W44,16-19,12651,217.0,"0,4",34664,595.0,2021-11-07
810,2021-W44,20-39,13792,273.0,"5,2",204320,4046.0,2021-11-07
811,2021-W44,40-64,17148,240.0,"5,6",324182,4538.0,2021-11-07
812,2021-W44,65-79,9166,153.0,"10,5",82773,1382.0,2021-11-07
813,2021-W44,80+,5919,105.0,"23,6",17283,306.0,2021-11-07


0.14456718926385567
0.26280960484023447
